In [59]:
# Import necessary modules from your files
import os
import subprocess
import numpy as np

from tqdm.notebook import tqdm

from searchclient.agent_types.classic import * 

# Import all action classes (used for hardcoding solutions) and actions libraries
from searchclient.domains.hospital.actions import (
    NoOpAction, MoveAction, PushAction, PullAction, AnyAction, DEFAULT_MAPF_ACTION_LIBRARY, DEFAULT_HOSPITAL_ACTION_LIBRARY
)

# Import state, goal description and level classes for the MAvis hospital environment
from searchclient.domains.hospital.state import HospitalState
from searchclient.domains.hospital.goal_description import HospitalGoalDescription
from searchclient.domains.hospital.level import HospitalLevel

# Import the Graph-Search algorithm
from searchclient.search_algorithms.graph_search import graph_search

# Import the different search strategies for both uninformed and informed search
from searchclient.strategies.bfs import FrontierBFS
from searchclient.strategies.dfs import FrontierDFS
from searchclient.strategies.bestfirst import FrontierBestFirst, FrontierGreedy, FrontierAStar

# Import heuristic classes, to be used in informed search methods
from searchclient.domains.hospital.heuristics import (
    HospitalZeroHeuristic, HospitalGoalCountHeuristics, HospitalAdvancedHeuristics
)

# Ensure the environment is set up properly
print("Modules imported successfully.")

Modules imported successfully.


In [60]:
# Function to load a level file
def load_level_file_from_path(path):
    with open(path, "r") as f:
        lines = f.readlines()
        lines = list(map(lambda line: line.strip(), lines))
        return lines
  
# Example usage: load_level('path_to_level_file.lvl')
level_path = "levels/SimpleDebug.lvl"
level_lines = load_level_file_from_path(level_path)
level = HospitalLevel.parse_level_lines(level_lines)

# We can access the initial state of the level using the following code
initial_state = HospitalState(level, level.initial_agent_positions, level.initial_box_positions)

# We can access the goal description of the level using the following code
goal_description = HospitalGoalDescription(level, level.box_goals + level.agent_goals)

print('The initial state of the level is:')
print(initial_state)

print('\nThe goal description of the level is:')
print(goal_description) # which tells us where the level objects (like boxes and agents) should be placed to satisfy the goal
print('\nSo agent zero starts at {} and satisfies the goal at {}'.format(level.initial_agent_positions[0][0], goal_description.agent_goals[0][0]))

The initial state of the level is:
++++++++
+      +
+      +
+ 0    +
+      +
+      +
+      +
++++++++

The goal description of the level is:
((5, 5), '0', True)

So agent zero starts at (3, 2) and satisfies the goal at (5, 5)


In [61]:
def render_plan(level_path, plan, strategy_name, heuristic_name, num_generated, elapsed_time, sol_length):

    str_plan = convert_plan_to_string(plan) #convert the plan to a string

    # this just makes sure that the meta information is displayed correctly in the visualization
    if strategy_name == 'greedy' or strategy_name == 'astar':
        strategy_name_pygame = strategy_name + ' w. ' + heuristic_name
    else:
        strategy_name_pygame = strategy_name
    
    subprocess.run(["python3", 
                    "renderMAvis.py", 
                    "--level", level_path, 
                    "--plan", str_plan, 
                    "--search_strategy", strategy_name_pygame, 
                    "--num_generated", str(num_generated), 
                    "--time_elapsed", str(elapsed_time), 
                    "--sol_length", str(sol_length)])

In [62]:
# Before running the search algorithm, we need to define the action set and the action library
# Use this library for pure pathfinding problems
action_library = DEFAULT_MAPF_ACTION_LIBRARY

# Use this library for sokoban-like problems (includes Push and Pull actions)
#action_library = DEFAULT_HOSPITAL_ACTION_LIBRARY

# Every agent will have the same action set
action_set = [action_library] * level.num_agents

# In order to run Graph-Search, we need to specify the initial state, action set, goal description, and frontier 

# If needed, we need to specify and fetch a heuristic function before initializing the frontier (informed search) 
# When adding new heuristics, remember to update the dictionary! 
# Use a string that matches the name of the heuristic function in the heuristics.py file
heuristic_name = "advanced" 
heuristic = {
        'zero': HospitalZeroHeuristic,
        'goalcount': HospitalGoalCountHeuristics,
        'advanced': HospitalAdvancedHeuristics,
    }.get(heuristic_name, HospitalAdvancedHeuristics)() # make sure you understand what .get() does


# Finally, let's pick the search strategy and fetch the relevant frontier
strategy_name = "greedy" 
frontier = {
        'bfs': FrontierBFS,
        'dfs': FrontierDFS,
        'astar': lambda: FrontierAStar(heuristic),
        'greedy': lambda: FrontierGreedy(heuristic)
    }.get(strategy_name, FrontierBFS)() # make sure you understand what .get() does

In [63]:
# Now that we have defined the initial state, action set, goal description, and frontier, we can run the search algorithm
planning_success, plan, num_generated, elapsed_time = graph_search(initial_state, action_set, goal_description, frontier)


# The graph search function returns the following:
print('Planning successful:', planning_success)
print('Plan:', plan)
print('Solution length:', len(plan))
print('Number of states generated:', num_generated)
print('Elapsed time:', elapsed_time)

Planning successful: True
Plan: [[Move(E)], [Move(S)], [Move(S)], [Move(E)], [Move(E)]]
Solution length: 5
Number of states generated: 6
Elapsed time: 0.0
